In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json, requests, pprint

In [2]:
food = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/all_restaurants2.csv',  index_col=[0])


In [3]:
food.head()

,country,city,name,formatted_address,price_level,rating,user_ratings_total,types,geometry.location.lat,geometry.location.lng,id,place_id
0,Thailand,Bangkok,Prachak Roasted Duck,"1415 Charoen Krung Rd, Silom, Bang Rak, Bangko...",1.0,4.2,2123,"['restaurant', 'food', 'point_of_interest', 'e...",13.720393,100.515971,db2193a7a92dbfff4cbdbb0f952cdb1990603975,ChIJVR1_rsWY4jAR0BqRhCK_sbM
2,Thailand,Bangkok,Best of the Boat Noodle,"2 18 16 Phahonyothin Soi 13, Samsen Nai, Phaya...",1.0,4.2,327,"['restaurant', 'food', 'point_of_interest', 'e...",13.765637,100.539628,da8d69669aadf454e456d7ba7e2ccd5d44ddaf35,ChIJLzmeirCe4jAR0LnyhkhTW6Q
4,Thailand,Bangkok,Sukhumvit Soi 38 - Street Food Court,"800 Sukhumvit Rd, Phra Khanong, Khlong Toei, B...",1.0,4.2,513,"['restaurant', 'food', 'point_of_interest', 'e...",13.723711,100.578966,0ec88b937253bac9d6484f22fe71c53a7b3dfd9e,ChIJE7hf3Kif4jARVENgIZ3Gy5o
5,Thailand,Bangkok,Chamlong's,"586 26 586/26 Kamphaeng Phet 1 Alley, Chatucha...",1.0,4.4,329,"['restaurant', 'food', 'point_of_interest', 'e...",13.799389,100.547041,eb56f64ff6556e065127792626c3c7941e4df2a2,ChIJ-cSOfBSc4jARX1N97kUbF4k
6,Thailand,Bangkok,Nai Mong Hoi Thod,"539 Phlap Phla Chai Rd, Pom Prap, Pom Prap Sat...",1.0,4.1,577,"['restaurant', 'food', 'point_of_interest', 'e...",13.742205,100.510777,b0ed69c726646b625fdc415245ff7e581e622120,ChIJb73JxSOZ4jARHgl4zhyCrLE


In [4]:
#Create a new column with the average
food['avg_rating'] = food.groupby(['country','city', 'price_level'])['rating'].transform('mean')


In [5]:
# Create a new city dataframe to add different aggregates to
city_df = food.groupby(['country','city', 'price_level'])['rating'].mean().to_frame()


In [6]:
# Create new columns in city_df
city_df['above_4.5_price'] = food[food['rating'] >= 4.5].groupby(['country', 'city', 'price_level'])['name'].count()
city_df['restaurant_count'] = food.groupby(['country', 'city', 'price_level'])['name'].count()
city_df['above_4.5_all'] = food[food['rating'] >= 4.5].groupby(['country', 'city'])['name'].count()
city_df['price_average_all'] = food.groupby(['country', 'city'])['price_level'].mean()


In [7]:
city_df

rating  above_4.5_price  \
country        city     price_level                              
Thailand       Bangkok  1.0          4.287500                6   
                        2.0          4.296429               14   
                        3.0          4.422727               11   
                        4.0          4.400000                6   
United Kingdom London   1.0          4.391379               24   
                        2.0          4.437349               40   
                        3.0          4.394118               28   
                        4.0          4.500000               22   
United States  Honolulu 1.0          4.473171               26   
                        2.0          4.390278               30   
                        3.0          4.478261               16   
                        4.0          4.490000                6   

                                     restaurant_count  above_4.5_all  \
country        city     price_level                                    
Thailand       Bangkok  1.0                        32             45   
                        2.0                        56             45   
                        3.0                        22             45   
                        4.0                        11             45   
United Kingdom London   1.0                        58            117   
                        2.0                        83            117   
                        3.0                        68            117   
                        4.0                        29            117   
United States  Honolulu 1.0                        41             82   
                        2.0                        72             82   
                        3.0                        23             82   
                        4.0                        10             82   

                                     price_average_all  
country        city     price_level                     
Thailand       Bangkok  1.0                   2.099174  
                        2.0                   2.099174  
                        3.0                   2.099174  
                        4.0                   2.099174  
United Kingdom London   1.0                   2.285714  
                        2.0                   2.285714  
                        3.0                   2.285714  
                        4.0                   2.285714  
United States  Honolulu 1.0                   2.013699  
                        2.0                   2.013699  
                        3.0                   2.013699  
                        4.0                   2.013699

In [8]:
describe_df = food.groupby(['country', 'city', 'price_level'])['rating'].describe()

In [9]:
city_df = pd.merge(city_df[['above_4.5_price', 'above_4.5_all']], describe_df, on= ['country', 'city', 'price_level'])

In [10]:
city_df

above_4.5_price  above_4.5_all  count  \
country        city     price_level                                          
Thailand       Bangkok  1.0                        6             45   32.0   
                        2.0                       14             45   56.0   
                        3.0                       11             45   22.0   
                        4.0                        6             45   11.0   
United Kingdom London   1.0                       24            117   58.0   
                        2.0                       40            117   83.0   
                        3.0                       28            117   68.0   
                        4.0                       22            117   29.0   
United States  Honolulu 1.0                       26             82   41.0   
                        2.0                       30             82   72.0   
                        3.0                       16             82   23.0   
                        4.0                        6             82   10.0   

                                         mean       std  min    25%   50%  \
country        city     price_level                                         
Thailand       Bangkok  1.0          4.287500  0.236575  3.6  4.175  4.30   
                        2.0          4.296429  0.195368  3.7  4.175  4.30   
                        3.0          4.422727  0.182396  3.9  4.300  4.45   
                        4.0          4.400000  0.219089  4.0  4.350  4.50   
United Kingdom London   1.0          4.391379  0.205439  3.9  4.300  4.40   
                        2.0          4.437349  0.148743  3.9  4.300  4.40   
                        3.0          4.394118  0.167405  3.7  4.300  4.40   
                        4.0          4.500000  0.143925  4.1  4.500  4.50   
United States  Honolulu 1.0          4.473171  0.196245  4.0  4.400  4.50   
                        2.0          4.390278  0.177748  3.8  4.300  4.40   
                        3.0          4.478261  0.073587  4.3  4.400  4.50   
                        4.0          4.490000  0.144914  4.3  4.400  4.50   

                                       75%  max  
country        city     price_level              
Thailand       Bangkok  1.0          4.400  4.8  
                        2.0          4.425  4.6  
                        3.0          4.600  4.6  
                        4.0          4.550  4.6  
United Kingdom London   1.0          4.500  4.8  
                        2.0          4.500  4.7  
                        3.0          4.500  4.7  
                        4.0          4.600  4.7  
United States  Honolulu 1.0          4.600  4.7  
                        2.0          4.500  4.6  
                        3.0          4.500  4.6  
                        4.0          4.575  4.7

In [11]:
food.groupby(['country', 'city', 'price_level'])['rating'].describe()

count      mean       std  min    25%  \
country        city     price_level                                          
Thailand       Bangkok  1.0           32.0  4.287500  0.236575  3.6  4.175   
                        2.0           56.0  4.296429  0.195368  3.7  4.175   
                        3.0           22.0  4.422727  0.182396  3.9  4.300   
                        4.0           11.0  4.400000  0.219089  4.0  4.350   
United Kingdom London   1.0           58.0  4.391379  0.205439  3.9  4.300   
                        2.0           83.0  4.437349  0.148743  3.9  4.300   
                        3.0           68.0  4.394118  0.167405  3.7  4.300   
                        4.0           29.0  4.500000  0.143925  4.1  4.500   
United States  Honolulu 1.0           41.0  4.473171  0.196245  4.0  4.400   
                        2.0           72.0  4.390278  0.177748  3.8  4.300   
                        3.0           23.0  4.478261  0.073587  4.3  4.400   
                        4.0           10.0  4.490000  0.144914  4.3  4.400   

                                      50%    75%  max  
country        city     price_level                    
Thailand       Bangkok  1.0          4.30  4.400  4.8  
                        2.0          4.30  4.425  4.6  
                        3.0          4.45  4.600  4.6  
                        4.0          4.50  4.550  4.6  
United Kingdom London   1.0          4.40  4.500  4.8  
                        2.0          4.40  4.500  4.7  
                        3.0          4.40  4.500  4.7  
                        4.0          4.50  4.600  4.7  
United States  Honolulu 1.0          4.50  4.600  4.7  
                        2.0          4.40  4.500  4.6  
                        3.0          4.50  4.500  4.6  
                        4.0          4.50  4.575  4.7

In [12]:
# Bin counts city-wide
bin_counts = food.groupby(['country', 'city'])['rating'].value_counts(bins=4).to_frame()

# bin counts city wide, per price
bin_counts_by_price = food.groupby(['country', 'city', 'price_level'])['rating'].value_counts(bins=4).to_frame()

In [14]:
bin_counts

rating
country        city                                       
Thailand       Bangkok  (4.3, 4.65]                     70
                        (3.95, 4.3]                     67
                        (4.65, 5.0]                      5
                        (3.5980000000000003, 3.95]       4
United Kingdom London   (4.25, 4.525]                  154
                        (4.525, 4.8]                    55
                        (3.975, 4.25]                   33
                        (3.698, 3.975]                   3
United States  Honolulu (4.3, 4.55]                     72
                        (4.55, 4.8]                     41
                        (4.05, 4.3]                     32
                        (3.798, 4.05]                    6